In [1]:
import tensorflow as tf
import keras
from keras import layers
from keras import models
from keras import utils
from keras.layers import Dense
from keras.models import Sequential
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Activation
from keras.regularizers import l2
from keras.optimizers import SGD
from keras.optimizers import RMSprop
from keras import datasets

from keras.callbacks import LearningRateScheduler
from keras.callbacks import History

from keras import losses
from sklearn.utils import shuffle
from keras.layers import Dense    #for Dense layers
from keras.layers import BatchNormalization #for batch normalization
from keras.layers import Dropout            #for random dropout
from keras.models import Sequential #for sequential implementation
from keras.optimizers import Adam   #for adam optimizer
from keras import regularizers      #for l2 regularization
from keras.wrappers.scikit_learn import KerasRegressor 
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import KFold 
from sklearn.model_selection import cross_val_score


Using TensorFlow backend.


In [2]:
import numpy as np
# fix random seed for reproducibility
np.random.seed(5)

In [3]:
# Importing and parsing Dataset 1
# Data Set:- FD001
# Train trjectories:- 100
# Test trajectories:- 100
# Conditions:- ONE (Sea Level)
# Fault Modes:- ONE (HPC Degradation)
import pandas as pd
import numpy as np
data_train_1 = pd.read_csv('C:/Users/Amardeep/Downloads/ML Nanodegree/PROJECT3/CMAPSSData/train_FD001.txt', delimiter=' ', header=None)
data_train_1.columns = ["unit number", "time_cycles", "op_set_1", "op_set_2","op_set_3","sm_1","sm_2","sm_3","sm_4","sm_5","sm_6","sm_7","sm_8","sm_9","sm_10","sm_11","sm_12","sm_13","sm_14","sm_15","sm_16","sm_17","sm_18","sm_19","sm_20","sm_21","sm_22","sm_23"]
data_train_1=data_train_1.drop(columns=['sm_22','sm_23'])

In [4]:
data_test_1 = pd.read_csv('C:/Users/Amardeep/Downloads/ML Nanodegree/PROJECT3/CMAPSSData/test_FD001.txt', sep=" ", header=None)
data_test_1.columns = ["unit number", "time_cycles", "op_set_1", "op_set_2","op_set_3","sm_1","sm_2","sm_3","sm_4","sm_5","sm_6","sm_7","sm_8","sm_9","sm_10","sm_11","sm_12","sm_13","sm_14","sm_15","sm_16","sm_17","sm_18","sm_19","sm_20","sm_21","sm_22","sm_23"]
data_test_1=data_test_1.drop(columns=['sm_22','sm_23'])

In [5]:
#Importing RULs for all datasets
RUL_1=pd.read_csv('C:/Users/Amardeep/Downloads/ML Nanodegree/PROJECT3/CMAPSSData/RUL_FD001.txt', sep=" ", header=None)
RUL_1.columns = ["RUL","d"]
RUL_1=RUL_1.drop(columns=['d'])
RUL_2=pd.read_csv('C:/Users/Amardeep/Downloads/ML Nanodegree/PROJECT3/CMAPSSData/RUL_FD002.txt', sep=" ", header=None)
RUL_2.columns = ["RUL","d"]
RUL_2=RUL_2.drop(columns=['d'])
RUL_3=pd.read_csv('C:/Users/Amardeep/Downloads/ML Nanodegree/PROJECT3/CMAPSSData/RUL_FD003.txt', sep=" ", header=None)
RUL_3.columns = ["RUL","d"]
RUL_3=RUL_3.drop(columns=['d'])
RUL_4=pd.read_csv('C:/Users/Amardeep/Downloads/ML Nanodegree/PROJECT3/CMAPSSData/RUL_FD004.txt', sep=" ", header=None)
RUL_4.columns = ["RUL","d"]
RUL_4=RUL_4.drop(columns=['d'])

In [6]:
#Make new column for RUL to match it with the test dataframe.
RUL_1['unit number']= 1+RUL_1['RUL'].index.values
#Merging common columns based on unit number.
data_test_1=pd.merge(data_test_1, RUL_1, on='unit number')

In [7]:
#Define a function to create RUL values for the training set.
def create_RUL(data, factor = 0):
    df = data.copy()
    RUL = df.groupby('unit number')['time_cycles'].max().reset_index()
    RUL = pd.DataFrame(RUL)
    RUL.columns = ['unit number','max']
    df = df.merge(RUL, on=['unit number'], how='left')
    df['RUL'] = df['max'] - df['time_cycles']
    df.drop(columns=['max'],inplace = True)
    
    return df[df['time_cycles'] > factor]

In [8]:
df_train= create_RUL(data_train_1)

In [9]:
#Loading features generated from training set:
wvt_train=pd.read_csv(r'C:\Users\Amardeep\Downloads\ML Nanodegree\PROJECT3\WVT\train\trainwvt1.csv')
wpsd_train=pd.read_csv(r'C:\Users\Amardeep\Downloads\ML Nanodegree\PROJECT3\WVT\train\trainwpsd1.csv')
opsd_train=pd.read_csv(r'C:\Users\Amardeep\Downloads\ML Nanodegree\PROJECT3\WVT\train\trainopsd1.csv')
vf_train=pd.read_csv(r'C:\Users\Amardeep\Downloads\ML Nanodegree\PROJECT3\WVT\train\trainvf1.csv')
fft_train=pd.read_csv(r'C:\Users\Amardeep\Downloads\ML Nanodegree\PROJECT3\WVT\train\fft_train_1.csv')
ae_train=pd.read_csv(r'C:\Users\Amardeep\Downloads\ML Nanodegree\PROJECT3\WVT\train\ae_train_1.csv')
ip_train=pd.read_csv(r'C:\Users\Amardeep\Downloads\ML Nanodegree\PROJECT3\WVT\train\ip_train_1.csv')
iff_train=pd.read_csv(r'C:\Users\Amardeep\Downloads\ML Nanodegree\PROJECT3\WVT\train\iff_train_1.csv')
tempshift_train=pd.read_csv(r'C:\Users\Amardeep\Downloads\ML Nanodegree\PROJECT3\WVT\train\tempshift_train_1.csv')
srtempshift_train=pd.read_csv(r'C:\Users\Amardeep\Downloads\ML Nanodegree\PROJECT3\WVT\train\srtempshift_train_1.csv')
rollshift_train=pd.read_csv(r'C:\Users\Amardeep\Downloads\ML Nanodegree\PROJECT3\WVT\train\rollshift_train_1.csv')
ac_train=pd.read_csv(r'C:\Users\Amardeep\Downloads\ML Nanodegree\PROJECT3\WVT\train\ac_train_1.csv')
cwtt_train=pd.read_csv(r'C:\Users\Amardeep\Downloads\ML Nanodegree\PROJECT3\WVT\train\cwtt_train_1.csv')
idwt_train=pd.read_csv(r'C:\Users\Amardeep\Downloads\ML Nanodegree\PROJECT3\WVT\train\idwt_train_1.csv')
lpf_train=pd.read_csv(r'C:\Users\Amardeep\Downloads\ML Nanodegree\PROJECT3\WVT\train\lpf_train_1.csv')
#Loading features generated from testing set:
wvt_test=pd.read_csv(r'C:\Users\Amardeep\Downloads\ML Nanodegree\PROJECT3\WVT\testwvt1.csv')
wpsd_test=pd.read_csv(r'C:\Users\Amardeep\Downloads\ML Nanodegree\PROJECT3\WVT\testwpsd1.csv')
opsd_test=pd.read_csv(r'C:\Users\Amardeep\Downloads\ML Nanodegree\PROJECT3\WVT\testopsd1.csv')
vf_test=pd.read_csv(r'C:\Users\Amardeep\Downloads\ML Nanodegree\PROJECT3\WVT\testvf1.csv')
fft_test=pd.read_csv(r'C:\Users\Amardeep\Downloads\ML Nanodegree\PROJECT3\WVT\fft_test_1.csv')
ae_test=pd.read_csv(r'C:\Users\Amardeep\Downloads\ML Nanodegree\PROJECT3\WVT\ae_test_1.csv')
ip_test=pd.read_csv(r'C:\Users\Amardeep\Downloads\ML Nanodegree\PROJECT3\WVT\ip_test_1.csv')
iff_test=pd.read_csv(r'C:\Users\Amardeep\Downloads\ML Nanodegree\PROJECT3\WVT\iff_test_1.csv')
tempshift_test=pd.read_csv(r'C:\Users\Amardeep\Downloads\ML Nanodegree\PROJECT3\WVT\tempshift_test_1.csv')
srtempshift_test=pd.read_csv(r'C:\Users\Amardeep\Downloads\ML Nanodegree\PROJECT3\WVT\srtempshift_test_1.csv')
rollshift_test=pd.read_csv(r'C:\Users\Amardeep\Downloads\ML Nanodegree\PROJECT3\WVT\rollshift_test_1.csv')
ac_test=pd.read_csv(r'C:\Users\Amardeep\Downloads\ML Nanodegree\PROJECT3\WVT\ac_test_1.csv')
cwtt_test=pd.read_csv(r'C:\Users\Amardeep\Downloads\ML Nanodegree\PROJECT3\WVT\cwtt_test_1.csv')
idwt_test=pd.read_csv(r'C:\Users\Amardeep\Downloads\ML Nanodegree\PROJECT3\WVT\idwt_test_1.csv')
lpf_test=pd.read_csv(r'C:\Users\Amardeep\Downloads\ML Nanodegree\PROJECT3\WVT\lpf_test_1.csv')

In [10]:
#Combining features with training set
new_df_train = pd.concat([df_train, wvt_train,wpsd_train, opsd_train,vf_train,fft_train,ae_train,ip_train,iff_train,tempshift_train,srtempshift_train,rollshift_train,cwtt_train,idwt_train,lpf_train], axis=1)
#Combining features with testing set
new_df_test= pd.concat([data_test_1, wvt_test,wpsd_test, opsd_test,vf_test,fft_test,ae_test,ip_test,iff_test,tempshift_test,srtempshift_test,rollshift_test,cwtt_test,idwt_test,lpf_test], axis=1)

In [11]:
#Dropping NaN values and filling remaining values with 0
new_df_train=new_df_train.dropna(how="all")
new_df_train=new_df_train.fillna(0)


In [12]:
#Filling NaN values in both training and testing datasets
new_df_train=new_df_train.fillna(0)
new_df_test=new_df_test.fillna(0)

In [13]:
X_train=new_df_train.drop(columns=['RUL'])
y_train=new_df_train['RUL']
X_test=new_df_test.drop(columns=['RUL'])
y_test=new_df_test['RUL']

In [34]:
#Remove uncommon column in both datasets
#Create separate lists for training and testing dataset columns.
traincolumns=X_train.columns
testcolumns=X_test.columns
trainc=traincolumns.values.tolist()
testc=testcolumns.values.tolist()
def Diff(li1, li2): 
    li_dif = [i for i in li1 + li2 if i not in li1 or i not in li2] 
    return li_dif 
  
# Driver Code 
list1 = Diff(testc, trainc) 
print(list1) 

[]


In [18]:
X_test=X_test.drop(columns=['fft_test_1', 'cwt_op_Set_1'])

In [19]:
#Confirm the shape of the generated new dataframes.
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(20632, 277)
(53760, 277)
(20632,)
(53760,)


In [14]:
from sklearn.model_selection import train_test_split
seed = 1
test_size=0.2
X_train, X_test, y_train, y_test = train_test_split(X_test, y_test, test_size=test_size, random_state=seed)

In [15]:
print(f'There are {X_train.shape[0]} train samples')
print(f'There are {X_test.shape[0]} test samples')

There are 43008 train samples
There are 10752 test samples


In [16]:
train_inputs = X_test.values
train_targets = y_test.values
print(train_inputs.shape)
print(train_targets.shape)

(10752, 279)
(10752,)


In [22]:
n_cols = train_inputs.shape[1]
input_shape = (n_cols, )
# Creates a model given an activation and learning rate
def create_model(learning_rate = 0.01, activation = 'relu'):
  
    # Create an Adam optimizer with the given learning rate
    opt = Adam(lr=learning_rate)
  
    #l  
    model = Sequential()
    model.add(Dense(128, 
                    activation = activation,
                    input_shape = input_shape,
                    activity_regularizer = regularizers.l2(1e-5)))
    model.add(Dropout(0.50))
    model.add(Dense(128,
                    activation = activation, 
                    activity_regularizer = regularizers.l2(1e-5)))
    model.add(Dropout(0.50))
    model.add(Dense(1, activation = activation))
# Compile the model
    model.compile(optimizer = opt,
                  loss = "mean_absolute_error",
                  metrics=['mse', "mape"])
    return model

In [23]:
# Create a KerasClassifier object
model = KerasRegressor(build_fn = create_model,
                       verbose = 0)
# Define the hyperparameter space
params = {'activation': ["relu", "tanh"],
          'batch_size': [16, 32, 64], 
          'epochs': [50, 100],
          'learning_rate': [0.01, 0.001, 0.0001]}
# Create a randomize search cv object 
random_search = RandomizedSearchCV(model,
                                   param_distributions = params,
                                   cv = KFold(10))
random_search_results = random_search.fit(train_inputs, train_targets)

In [31]:
print("Best Score: ",
      random_search_results.best_score_,
      "and Best Params: ",
      random_search_results.best_params_)

Best Score:  -16.184173959346843 and Best Params:  {'learning_rate': 0.0001, 'epochs': 100, 'batch_size': 16, 'activation': 'tanh'}


In [32]:
n_cols = train_inputs.shape[1]
input_shape = (n_cols, )
# Create the model object with default arguments
def create_model(learning_rate = 0.0001, activation='tanh'):
  
    # Set Adam optimizer with the given learning rate
    opt = Adam(lr = learning_rate)
  
    # Create your binary classification model  
    model = Sequential()
    model.add(Dense(128,
                    activation = activation,
                    input_shape = input_shape,
                    activity_regularizer = regularizers.l2(1e-5)))
    model.add(Dropout(0.50))
    model.add(Dense(128,
                    activation = activation, 
                    activity_regularizer = regularizers.l2(1e-5)))
    model.add(Dropout(0.50))
    model.add(Dense(1, activation = activation))
    # Compile the model
    model.compile(optimizer = opt,
                  loss = "mean_absolute_error",
                  metrics = ['mse', "mape"])
    return model

In [33]:
#Reevaluating with best parameters
model = KerasRegressor(build_fn = create_model,
                       epochs = 100, 
                       batch_size = 16,
                       verbose = 0)
# Calculate the accuracy score for each fold
kfolds = cross_val_score(model,
                         train_inputs,
                         train_targets,
                         cv = 10)
# Print the mean accuracy
print('The mean accuracy was:', kfolds.mean())
# Print the accuracy standard deviation
print('With a standard deviation of:', kfolds.std())

The mean accuracy was: -16.183890022436934
With a standard deviation of: 0.863807563878001


In [34]:
model.sk_params

{'epochs': 100, 'batch_size': 16, 'verbose': 0}

In [36]:
model.filter_sk_params

<bound method BaseWrapper.filter_sk_params of <keras.wrappers.scikit_learn.KerasRegressor object at 0x0000024AB7DCF108>>

In [37]:
model.fit

<bound method BaseWrapper.fit of <keras.wrappers.scikit_learn.KerasRegressor object at 0x0000024AB7DCF108>>

In [43]:
model.score

<bound method KerasRegressor.score of <keras.wrappers.scikit_learn.KerasRegressor object at 0x0000024AB7DCF108>>

In [41]:
model.fit(X_test, y_test)

In [42]:
model.model.save('saved_kerasregressor_model.h5')

In [46]:
model.model.weights

[<tf.Variable 'dense_334/kernel:0' shape=(279, 128) dtype=float32, numpy=
 array([[ 0.10412333, -0.09738728, -0.00933898, ...,  0.00058707,
         -0.01583734, -0.17997298],
        [-0.09125005,  0.02386453, -0.03660853, ..., -0.01521669,
         -0.03074618, -0.12718363],
        [ 0.01966858,  0.06339759, -0.10620739, ...,  0.08978471,
         -0.12096994,  0.09963976],
        ...,
        [-0.02484225, -0.03704802, -0.05872082, ...,  0.04288492,
         -0.07967255,  0.10123626],
        [-0.00219815, -0.10300548,  0.07577722, ..., -0.08634397,
         -0.04395959, -0.05524709],
        [-0.07008818,  0.05625782, -0.06017948, ..., -0.11712108,
          0.09684663,  0.0223255 ]], dtype=float32)>,
 <tf.Variable 'dense_334/bias:0' shape=(128,) dtype=float32, numpy=
 array([ 1.99530255e-02, -5.64282946e-03,  8.97656195e-03,  3.62281944e-03,
        -8.48817732e-03, -1.13582006e-02,  3.80437938e-04, -1.09038940e-02,
        -1.82823986e-02, -7.07688602e-03,  1.61736328e-02,  1.7

In [48]:
model.model.loss_weights_list

[1.0]

In [49]:
model.model.metrics_names

['loss', 'mse', 'mape']

In [50]:
model.model.metrics

In [ ]:
# Use scikit-learn to grid search the dropout rate
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasRegressor
from keras.constraints import maxnorm
# Function to create model, required for KerasRegressor
def create_model(dropout_rate=0.0, weight_constraint=0):
	# create model
	model = Sequential()
	model.add(Dense(500, input_dim=279, kernel_initializer='uniform', activation='tanh', kernel_constraint=maxnorm(weight_constraint)))
	model.add(Dropout(dropout_rate))
	model.add(Dense(1, kernel_initializer='uniform', activation='tanh'))
	# Compile model
	model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
	return model
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# create model
model = KerasRegressor(build_fn=create_model, epochs=100, batch_size=16, verbose=0)
# define the grid search parameters
weight_constraint = [1, 2, 3, 4, 5]
dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
param_grid = dict(dropout_rate=dropout_rate, weight_constraint=weight_constraint)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))